In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('/home/briggsc1-erau.edu/Downloads/housing.csv')
features = ['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income']
target = ['median_house_value']
df = df.dropna(subset = features+target)

In [ ]:
x_tr,x_te,y_tr,y_te = train_test_split(df[features],df[target],
                                       test_size = 0.4, random_state = 0)
x_va,x_te,y_va,y_te = train_test_split(x_te,y_te,
                                       test_size = 0.5, random_state = 0)

### map

map takes a function and an iterable, applies the function to each thing in the iterable, and returns an iterator containing the results.

In [ ]:
def double(n):
    return(2*n)
nums = [1,2,3,4]
v = map(double, nums)
for x in v: print(x)

### functools.reduce

functools.reduce takes a function (of two arguments) and an iterable, applies the function successively along the iterable, and returns the result.

In [ ]:
from functools import reduce
def subtract(a,b):
    print(f'{a} minus {b} is {a-b}')
    return(a-b)
reduce(subtract,map(double,nums))

Warning: once you use an iterator once, it's used up.

Q: why does it not work to call reduce(sum,map(double,nums))?

You try: use map and reduce to get the sum of the squares up to n.

### Multiprocessing

Linux terminal: lscpu to see how many CPUs you have.

When multiprocessing, you should use at least CPUs - 1 processes. If you have 10 CPUs, use at least 9 processes. Reason: python is single-threaded. Terminal: top to see a process's id (PID), and cat /proc\/<proc id\>\/status to see the number of threads taken by a process.

In [ ]:
# toy example for multiprocessing: adding up the squares from 1 to N

from multiprocessing import Pool

N=100
def test(x):
    return(x**2)
inpt = range(1,N+1)
with Pool(processes = 19) as p:
    res = p.map(test,inpt)
sum(res)

The idea is that we create a pool of processes, then distribute (map) the function calls over the iterable input.

Here's an example where the function has two inputs. It adds up $a^b$ where $a$ ranges from 1 to 3 and $b$ ranges from 0 to 3.

In [ ]:
def test(x):
    return(x[0]**x[1])
inpt = product(range(1,4),range(4))
with Pool(processes = 10) as p:
    res = p.map(test,inpt)
sum(res)

Now we can apply this to the problem we had last time with the absolute_error trees taking a long time to train. It will still take a few minutes (depending on how good these lab machines are), but it will be doable.

In [ ]:
tr = DecisionTreeRegressor(random_state = 0)
tr.fit(x_tr,y_tr)
ud = tr.get_depth()

In [ ]:
trees = []
def fit_tree(inpt):
    max_depth,min_samples_leaf = inpt
    tr = DecisionTreeRegressor(max_depth = max_depth,
                               min_samples_leaf = min_samples_leaf,
                               random_state = 0,
                               criterion = 'absolute_error')
    tr.fit(x_tr,y_tr)
    return(tr)
inpt = product(range(1,ud+1),range(1,13))
with Pool(processes = 19) as p: # adjust processes according to your hardware
    trees = p.map(fit_tree,inpt)

Now let's get trees for the other three criteria (we did this last time).

In [ ]:
def fit_tree(inpt):
    max_depth,min_samples_leaf,crit = inpt
    tr = DecisionTreeRegressor(max_depth = max_depth,
                               min_samples_leaf = min_samples_leaf,
                               random_state = 0,
                               criterion = crit)
    tr.fit(x_tr,y_tr)
    return(tr)
inpt = product(range(1,ud+1),range(1,13),['squared_error','friedman_mse','poisson'])
with Pool(processes = 19) as p:
    trees2 = p.map(fit_tree,inpt)
trees = trees+trees2

Now make the dataframe of scores.

In [ ]:
data = []
for tree in trees:
    tr_sc = tree.score(x_tr,y_tr)
    va_sc = tree.score(x_va,y_va)
    data.append({'crit':tree.criterion,'set':'train','score':tr_sc,'num_leaves':tree.get_n_leaves(),
                 'depth':tree.max_depth,'min_samples':tree.min_samples_leaf})
    data.append({'crit':tree.criterion,'set':'val','score':va_sc,'num_leaves':tree.get_n_leaves(),
                 'depth':tree.max_depth,'min_samples':tree.min_samples_leaf})
df_acc = pd.DataFrame(data)
# create a multi-indexed dataframe
multi = df_acc.set_index(['set','crit','depth','min_samples'])
multi = multi.sort_index()

It would be difficult to usefully plot all four criteria at once. However, we can get the information we need to comparing the criteria pairwise.

In [ ]:
crit1 = 'squared_error' # the bigger dots
crit2 = 'absolute_error' # the smaller dots
fig,axes = plt.subplots(4,3,figsize = (15,9))
fig.tight_layout(pad=5.0)
for j in range(12):
    axes[j%4,j//4].scatter(range(depth_lower,ud+1),multi.loc[('train',crit1,slice(None),j+1)].score,
            color = '#FF0000')
    axes[j%4,j//4].scatter(range(depth_lower,ud+1),multi.loc[('val',crit1,slice(None),j+1)].score,
            color = '#00FF00')
    axes[j%4,j//4].scatter(range(depth_lower,ud+1),multi.loc[('train',crit2,slice(None),j+1)].score,
            color = '#770000',s = 10)
    axes[j%4,j//4].scatter(range(depth_lower,ud+1),multi.loc[('val',crit2,slice(None),j+1)].score,
            color = '#000077',s = 10)
    axes[j%4,j//4].set_title(f'min_samples_leaf = {j+1}')
    axes[j%4,j//4].grid()
    axes[j%4,j//4].set_xticks(range(0,ud+1,2))
    axes[j%4,j//4].set_ylim(0,1.1)
    axes[j%4,j//4].set_yticks(np.linspace(0,1,6))
plt.show();

It's not so easy to eyeball, so we can subset the scores dataframe with set == val, then sort by score.

In [ ]:
df_acc[(df_acc['set']=='val')&(df_acc['crit']!='absolute_error')].sort_values('score',ascending = False).head(10)

absolute_error appears to dominate. But the others aren't **that** far behind:

In [ ]:
print('Best, by criterion')
print(f'    score        criterion          max_depth        min_samples_leaf')
print('  '+'-'*63)
for crit in df_acc['crit'].unique():
    idx = df_acc[(df_acc['crit']==crit)&(df_acc['set']=='val')]['score'].idxmax()
    row = df_acc.loc[idx]
    
    print(f'  | {round(best,5):<10} | {row.crit:<16} | {row.depth:<14} | {row.min_samples:>2}')

### Bootstrapping

Now we discuss bootstrapping. Bootstrapping is a fundamental statistical technique, especially in nonparametric statistics. We use repeated sampling to empirically formulate estimates of population parameters without supposing a distribution.

In [ ]:
from sklearn.linear_model import LinearRegression
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
iris = pd.read_csv('https://gist.github.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv')
iris.head()

In [ ]:
intercepts = [] # to hold the y-intercepts of the lines of best fit
coefs = [] # to hold the slopes
# 100 iterations: sample the data, fit a line, and get the parameters
for i in range(100):
    choices = np.random.choice(range(150), size=50, replace=True)
    lr = LinearRegression()
    lr.fit(iris[['petal.width']].loc[choices],iris[['petal.length']].loc[choices])
    intercepts.append(lr.intercept_.item())
    coefs.append(lr.coef_.item())
# collect the parameter samples into a dataframe
reg_df = pd.DataFrame({'coef':coefs,'intercept':intercepts})

In [ ]:
# a function which will take a set of x-values and a dataframe with coef and intercept
# it will return the min and max y-values of all the lines for each x-value
def minmax_preds(xs,df):
    mins = []
    maxes = []
    for x in xs:
        x_max = df.iloc[0]['coef']*x+df.iloc[0]['intercept']
        x_min = df.iloc[0]['coef']*x+df.iloc[0]['intercept']
        for i,row in df.iterrows():
            y = row['coef']*x+row['intercept']
            if y<x_min:
                x_min = y
            if y>x_max:
                x_max = y
        mins.append(x_min)
        maxes.append(x_max)
    return(mins,maxes)

In [ ]:
# define the xs to be the range of petal.width values
xs = np.linspace(0,max(iris['petal.width']),50)
# get the min and max prediction of all 100 lines for each x-value
ymins,ymaxes = minmax_preds(xs,reg_df)

In [ ]:
# plot the lower and upper bounds of the line predictions for each x-value
fig,ax = plt.subplots(figsize = (6,4))
ax.scatter(xs,ymins,s=3,color = '#FF0000')
ax.scatter(xs,ymaxes,s=3,color = '#FF0000')
ax.scatter(iris['petal.width'],iris['petal.length'])
ax.set_title('Iris / lines of best fit 95% CI')
ax.set_xlabel('petal width')
ax.set_ylabel('petal length');

### Cross-validation

k-fold cross-validation is a sort of bootstrapping. It means after splitting off the test data from a dataset, breaking the remainder into k evenly-sized pieces. Each of the k pieces is, in turn, treated as a validation set, while the other k-1 pieces act as training sets. Doing so will give us a collection of parameters which can be averaged. This approach is less prone to overfit since it uses several training sets. Let's use k-fold cross validation on the iris set.

In [ ]:
from sklearn.model_selection import KFold, train_test_split

In [ ]:
x_tr,x_te,y_tr,y_te = train_test_split(iris[['petal.width']],iris[['petal.length']],
                                      test_size = 0.2,random_state = 0)

In [ ]:
kf = KFold(n_splits=3) # by default, the input will not be shuffled!
kf.get_n_splits(x_tr) # define splits on the x_tr indices
# kf.split(x_tr) returns a generator. let's investigate:
for i, (train_index, val_index) in enumerate(kf.split(x_tr)):
    print(f'FOLD {i}:')
    print(f'  TRAIN INDEX={train_index}')
    print(f'  VAL INDEX={val_index}')

In [ ]:
coefs = []
intercepts = []
for fold,(train_index,val_index) in enumerate(kf.split(x_tr)):
    lr.fit(x_tr.iloc[train_index],y_tr.iloc[train_index])
    coefs.append(lr.coef_)
    intercepts.append(lr.intercept_)

In [ ]:
coef = round(np.mean(coefs),3)
intercept = round(np.mean(intercepts),3)
print(f'coef: {coef}\nintercept: {intercept}')

In [ ]:
# plot the k-folds average line of best fit vs the 95% CI min/max
fig,ax = plt.subplots(figsize = (6,4))
ax.scatter(xs,ymins,s=3,color = '#FF0000',label = '95% CI min/max')
ax.scatter(xs,ymaxes,s=3,color = '#FF0000')
ax.scatter(iris['petal.width'],iris['petal.length'])
ax.plot(xs,[x*coef+intercept for x in xs],color = '#000000',label = 'k-fold avg')
ax.set_title('Iris / lines of best fit 95% CI')
ax.set_xlabel('petal width')
ax.set_ylabel('petal length')
ax.legend();

In [ ]:
# now with more folds!
folds = 10
kf = KFold(n_splits=folds)
kf.get_n_splits(x_tr)
coefs = []
intercepts = []
for fold,(train_index,val_index) in enumerate(kf.split(x_tr)):
    lr.fit(x_tr.iloc[train_index],y_tr.iloc[train_index])
    coefs.append(lr.coef_)
    intercepts.append(lr.intercept_)
fig,ax = plt.subplots(figsize = (6,4))
ax.scatter(xs,ymins,s=3,color = '#FF0000',label = '95% CI min/max')
ax.scatter(xs,ymaxes,s=3,color = '#FF0000')
ax.scatter(iris['petal.width'],iris['petal.length'])
ax.plot(xs,[x*coef+intercept for x in xs],color = '#000000',label = f'{folds}-fold avg')
ax.set_title('Iris / lines of best fit 95% CI')
ax.set_xlabel('petal width')
ax.set_ylabel('petal length')
ax.legend();